# Comparing the predictions from multiple models
---

I've trained three networks so far, all for 60 epochs, batch size 128:
- a simple network consisting of one `LSTM` layer with 128 units
- a simple network comprising one `LSTM` layer with 64 units
- a deeper network containing:
    - 128-unit `LSTM` + 0.8 `Dropout`
    - 64-unit `LSTM` + 0.8 `Dropout`
    - 32-unit `LSTM` + 0.8 `Dropout`
    
I want to generate sequences from all three of them and see how they compare, from maybe 3 seed sentences.

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import load_model

Prepare the text and load the saved models.

In [11]:
path = "infinite_jest_text.txt"

with open(path, "r") as f:
    text = f.read()
    
text = text.lower().replace("\n", " ")
chars = sorted(list(set(text)))

# We need to be able to convert from chars to indices and vice-versa.
char_to_idx = dict((c, i) for (i, c) in enumerate(chars))
idx_to_char = dict((i, c) for (i, c) in enumerate(chars))

In [19]:
# Saved models are detailed above.
model_names = ["64unit_60epochs", "128unit_60epochs", "deep_60epochs"]
models = {}

for model_name in model_names:
    models[model_name] = load_model("saved_models/" + model_name)

print(models)

{'64unit_60epochs': <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fa2e03277f0>, '128unit_60epochs': <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fa2d3f4db80>, 'deep_60epochs': <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fa3118d83a0>}


In [15]:
def sample(preds, temperature=1.0):
    """Sample from an array of predictions with a given temperature/diversity.
       Author: fchollet
    """
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    preds = np.exp(preds)
    preds /= np.sum(preds)
    return np.argmax(np.random.multinomial(1, preds, 1))

In [20]:
temperatures = [0.3, 0.7, 1.0]
sentlen = 40
seed_index = np.random.randint(len(text)-sentlen)
seed_sentence = text[seed_index : seed_index + sentlen]
out_path = "output.txt"

with open(out_path, "a") as out:
    out.write("Seed sentence: {}\n\n".format(seed_sentence))
    for name, model in models.items():
        out.write("#"*40 + "\n")
        out.write("Model: {}\n".format(name))
        for temperature in temperatures:
            out.write("_"*40 + "\n")
            out.write("Temp: {}\n".format(temperature))

            generated = ""
            for i in range(400):
                # Encode seed sentence.
                x_pred = np.zeros((1, sentlen, len(chars)))
                for char_index, char in enumerate(seed_sentence):
                    x_pred[0, char_index, char_to_idx[char]] = 1

                preds = model.predict(x_pred)[0]
                next_char = idx_to_char[sample(preds, temperature)]

                seed_sentence = seed_sentence[1:] + next_char
                generated += next_char
            out.write(generated + "\n")

<ipython-input-15-507ff383eb62>:6: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature
